In [114]:
import numpy as np
import pandas as pd
import requests, time
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings('ignore')

In [55]:
lines = []
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver', options=options)
url = 'https://www.melon.com/chart/'
driver.get(url)
time.sleep(2)
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tbody > tr')
for tr in tqdm(trs):
    driver.find_element(By.XPATH, '//*[@id="lst50"]/td[5]/div/a').click()
    time.sleep(2)
    image = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[1]/a/img').get_attribute('src')
    title = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[1]').text
    artist = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[2]/a/span[1]').text
    album = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[1]/a').text
    date = driver.find_element(By.XPATH, '//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
    lyric = driver.find_element(By.XPATH, '//*[@id="d_video_summary"]').text
    lines.append({'image': image, 'title': title, 'artist': artist, 'album': album, 'date':date,
                  'lyric': lyric.replace('\n', ' ').replace('(', '').replace(')', '')})
    driver.back()

df = pd.DataFrame(lines)
df.to_csv('data/melon_test.csv', index=False)

100%|██████████| 100/100 [07:15<00:00,  4.35s/it]


In [123]:
lines = []
url = 'https://www.melon.com/chart/'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
res = requests.get(url, headers=header)
soup = BeautifulSoup(res.text)
trs = soup.select('tbody > tr')
for i in tqdm(range(36406115, 36416115)):
    href = f'https://www.melon.com/song/detail.htm?songId={i}'
    request = requests.get(href, headers=header)
    soup = BeautifulSoup(request.text)
    info = soup.select_one('#conts')
    try:
        image = soup.select_one('.image_typeAll > img')['src']
    except:
        image = np.nan
    try:
        title = info.select_one('.song_name').get_text().strip().split('\t')[-1]
    except:
        title = np.nan
    try:
        artist = info.select_one('.artist_name').get_text()
    except:
        artist = np.nan
    try:
        album = info.select_one('.list > dd').get_text()
    except:
        album = np.nan
    try:
        date = info.select_one('dl > dd:nth-child(4)').get_text()
    except:
        date = np.nan
    try:
        genre = info.select_one('dl > dd:nth-child(6)').get_text()
    except:
        genre = np.nan
    try:
        lyric = info.select_one('.lyric').get_text().strip()
        lyric = lyric.replace('\n', ' ').replace('(', '').replace(')', '')
    except:
        lyric = np.nan
    lines.append({'image': image, 'title': title, 'artist': artist,
                  'album': album, 'date':date, 'genre':genre, 'lyric': lyric})

df = pd.DataFrame(lines)
df.dropna(subset=['title'], inplace=True)
df.to_csv('data/melon_test.csv', index=False)

100%|██████████| 10000/10000 [1:03:57<00:00,  2.61it/s]


In [120]:
df = pd.read_csv('data/melon_test.csv')
df.tail()

,image,title,artist,album,date,genre,lyric
95,https://cdnimg.melon.co.kr/cm/album/images/028...,Beetles (Snorkle Remix),Dunkan,Beetles,2012.11.30,일렉트로니카,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,https://cdnimg.melon.co.kr/cm/album/images/028...,East Bound Freight Train (Original Mix),Grandpa Jones,Bald Headed End of the Broom,2015.03.04,재즈,NaN
98,https://cdnimg.melon.co.kr/cm/album/images/028...,What Am I Gonna Do,Patrick Nuo,Nuo,2007.06.08,POP,NaN
99,https://cdnimg.melon.co.kr/cm/album/images/028...,Under Control,Giulio Lnt & Franx,"Ibiza Calling, Vol. 2 (Selected By Simon Lunardi)",2015.12.17,클래식,NaN


In [ ]:
10000~36416114